# パイプライン ジョブとしてスクリプトを実行する

パイプラインを使用すると、複数のステップを 1 つのワークフローにグループ化することができます。 コンポーネントを使用してパイプラインを構築できます。 各コンポーネントには、実行する Python スクリプトが反映されます。 コンポーネントは、スクリプトとその実行方法を指定する YAML ファイルで定義されます。 

## 開始する前に

このノートブックでコードを実行するには、最新バージョンの **azureml-ai-ml** パッケージが必要です。 次のセルを実行して、パッケージがインストールされていることを確認します。

> **注**:
> **azure-ai-ml** パッケージがインストールされていない場合は、`pip install azure-ai-ml` を実行してインストールします。

In [ ]:
## ワークスペースに接続する

必要な SDK パッケージがインストールされているため、ワークスペースに接続できます。

ワークスペースに接続するには、識別子パラメーター (サブスクリプション ID、リソース グループ名、ワークスペース名) が必要です。 リソース グループ名とワークスペース名は、既に入力されています。 コマンドを完了するには、サブスクリプション ID のみが必要です。

必要なパラメーターを見つけるには、スタジオの右上にあるサブスクリプションとワークスペース名をクリックします。 右側にペインが開きます。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> サブスクリプション ID をコピーし、**YOUR-SUBSCRIPTION-ID** をコピーした値に置き換えます。 </p>

## スクリプトを作成する

次の 2 つの手順でパイプラインを構築します。

1. **データを準備する**: 不足しているデータを修正し、データを正規化します。
1. **モデルのトレーニング**: デシジョン ツリー分類モデルをトレーニングします。

次のセルを実行して、**src** フォルダーと 2 つのスクリプトを作成します。

In [ ]:
## コンポーネントを定義する

コンポーネントを定義するには、次のように指定する必要があります。

- **メタデータ**: "名前"、"表示名"、"バージョン"、"説明"、"種類" など。メタデータは、コンポーネントの記述と管理に役立ちます。** ** ** ** **
- **インターフェイス**: "入力" と "出力"。** ** たとえば、モデル トレーニング コンポーネントは、トレーニング データと正則化率を入力として受け取り、トレーニング済みのモデル ファイルを出力として生成します。 
- **コマンド、コード、環境**: コンポーネントを実行するための "コマンド"、"コード"、"環境"。** ** ** コマンドは、コンポーネントを実行するためのシェル コマンドです。 コードは通常、ソース コード ディレクトリを参照します。 環境は、AzureML 環境 (キュレーションされた環境またはカスタム作成された環境)、Docker イメージ、または conda 環境の場合があります。

次のセルを実行して、パイプライン ステップとして実行するコンポーネントごとに YAML を作成します。

In [ ]:
## コンポーネントを読み込む

各コンポーネントを定義したら、YAML ファイルを参照してコンポーネントを読み込むことができます。 

## パイプラインを構築する

コンポーネントを作成して読み込むと、パイプラインを構築できます。 2 つのコンポーネントをパイプラインに作成します。 まず、`prep_data` コンポーネントを実行します。 最初のコンポーネントの出力は、モデルをトレーニングする 2 番目のコンポーネント `train_decision_tree` の入力である必要があります。

`diabetes_classification` 関数は、完全なパイプラインを表します。 この関数には 1 つの入力パラメーター `pipeline_job_input` が必要です。 セットアップ中にデータ資産が作成されました。 登録済みのデータ資産をパイプライン入力として使用します。 

In [ ]:
`pipeline_job` オブジェクトを印刷すると、パイプライン ジョブの構成を取得できます。

In [ ]:
パラメーターを参照し、新しい値を指定すると、パイプライン ジョブ構成の任意のパラメーターを変更できます。

In [ ]:
## パイプライン ジョブを送信する

最後に、パイプラインを構築し、必要に応じて実行するようにパイプライン ジョブを構成したら、パイプライン ジョブを送信できます。

## Define the components

To define the component you need to specify:

- **Metadata**: *name*, *display name*, *version*, *description*, *type* etc. The metadata helps to describe and manage the component.
- **Interface**: *inputs* and *outputs*. For example, a model training component will take training data and the regularization rate as input, and generate a trained model file as output. 
- **Command, code & environment**: the *command*, *code* and *environment* to run the component. Command is the shell command to execute the component. Code usually refers to a source code directory. Environment could be an AzureML environment (curated or custom created), docker image or conda environment.

Run the following cells to create a YAML for each component you want to run as a pipeline step.

In [ ]:
%%writefile prep-data.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: prep_data
display_name: Prepare training data
version: 1
type: command
inputs:
  input_data: 
    type: uri_file
outputs:
  output_data:
    type: uri_file
code: ./src
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
command: >-
  python prep-data.py 
  --input_data ${{inputs.input_data}}
  --output_data ${{outputs.output_data}}

In [ ]:
%%writefile train-model.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: train_model
display_name: Train a decision tree classifier model
version: 1
type: command
inputs:
  training_data: 
    type: uri_file
  reg_rate:
    type: number
    default: 0.01
outputs:
  model_output:
    type: mlflow_model
code: ./src
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
command: >-
  python train-model.py 
  --training_data ${{inputs.training_data}} 
  --reg_rate ${{inputs.reg_rate}} 
  --model_output ${{outputs.model_output}} 

## Load the components

Now that you have defined each component, you can load the components by referring to the YAML files. 

In [ ]:
from azure.ai.ml import load_component
parent_dir = ""

prep_data = load_component(source=parent_dir + "./prep-data.yml")
train_decision_tree = load_component(source=parent_dir + "./train-model.yml")

## Build the pipeline

After creating and loading the components, you can build the pipeline. You'll compose the two components into a pipeline. First, you'll want the `prep_data` component to run. The output of the first component should be the input of the second component `train_decision_tree`, which will train the model.

The `diabetes_classification` function represents the complete pipeline. The function expects one input variable: `pipeline_job_input`. A data asset was created during setup. You'll use the registered data asset as the pipeline input. 

In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline

@pipeline()
def diabetes_classification(pipeline_job_input):
    clean_data = prep_data(input_data=pipeline_job_input)
    train_model = train_decision_tree(training_data=clean_data.outputs.output_data)

    return {
        "pipeline_job_transformed_data": clean_data.outputs.output_data,
        "pipeline_job_trained_model": train_model.outputs.model_output,
    }

pipeline_job = diabetes_classification(Input(type=AssetTypes.URI_FILE, path="azureml:diabetes-data:1"))

You can retrieve the configuration of the pipeline job by printing the `pipeline_job` object:

In [ ]:
print(pipeline_job)

You can change any parameter of the pipeline job configuration by referring to the parameter and specifying the new value:

In [ ]:
# change the output mode
pipeline_job.outputs.pipeline_job_transformed_data.mode = "upload"
pipeline_job.outputs.pipeline_job_trained_model.mode = "upload"
# set pipeline level compute
pipeline_job.settings.default_compute = "aml-cluster"
# set pipeline level datastore
pipeline_job.settings.default_datastore = "workspaceblobstore"

# print the pipeline job again to review the changes
print(pipeline_job)

## Submit the pipeline job

Finally, when you've built the pipeline and configured the pipeline job to run as required, you can submit the pipeline job:

In [ ]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_diabetes"
)
pipeline_job